In [1]:
import ray, random, os 
from ray.air.config import ScalingConfig
import pandas as pd
ray.init(_temp_dir='/Volumes/SSD980/ray')
from ray import air, tune
from ray.tune.schedulers import PopulationBasedTraining
import argparse
from ray.tune import CLIReporter
from worker_standlone import WorkerStandAlone
from multi_agent import MultiAgent
from manager import Manager
from ray.train.rl import RLTrainer
from ray.rllib.policy.policy import Policy, PolicySpec
from ray.tune.registry import register_env
from ray.tune import TuneConfig
from ray.tune.logger import pretty_print
from ray.tune.search.bayesopt import BayesOptSearch
from ray.air.config import RunConfig, CheckpointConfig
from ray.tune.schedulers import HyperBandScheduler
from ray.rllib.algorithms.ppo import PPOConfig
from ray.tune.search.hyperopt import HyperOptSearch
from ray.tune.schedulers import ASHAScheduler
from ray.rllib.utils import check_env

2023-09-29 10:20:30,371	INFO worker.py:1612 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8266 
2023-09-29 10:20:37,018	WARNING deprecation.py:50 -- DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!


In [2]:
from ray.rllib.utils import check_env
check_env(MultiAgent())

ValueError: Traceback (most recent call last):
  File "/Users/floriankockler/anaconda3/envs/py310/lib/python3.10/site-packages/ray/rllib/utils/pre_checks/env.py", line 363, in check_multiagent_environments
    results = env.step(sampled_action)
  File "/Users/floriankockler/Documents/GitHub.nosync/quantumai4/multi_agent.py", line 90, in step
    agent = self.agents[agent_id]
KeyError: 'controller'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/floriankockler/anaconda3/envs/py310/lib/python3.10/site-packages/ray/rllib/utils/pre_checks/env.py", line 81, in check_env
    check_multiagent_environments(env)
  File "/Users/floriankockler/anaconda3/envs/py310/lib/python3.10/site-packages/ray/rllib/utils/pre_checks/env.py", line 368, in check_multiagent_environments
    raise ValueError(
ValueError: Your environment (<MultiAgent instance>) does not abide to the new gymnasium-style API!
From Ray 2.3 on, RLlib only support

In [3]:
multi_agent_env = MultiAgent()


for episode in range(1):
    obs, info = multi_agent_env.reset()
    done = {"__all__": False}
    
    print(f"Episode {episode + 1}")

    while not done["__all__"]:
        actions = {}
        
        # Collect actions for each agent
        for agent_id, agent_obs in obs.items():
            action_space = multi_agent_env.action_space[agent_id]
            action = action_space.sample()
            actions[agent_id] = action

        # Step the environment
        obs, reward, done, _, info = multi_agent_env.step(actions)

        # print(f"Actions: {actions}")
        # print(f"Observations: {obs}")
        # print(f"Rewards: {reward}")
        # print(f"Done flags: {done}")
        # print(f"Info: {info}")

    print("Episode done!")

Episode 1
reward MA {'ABT.US': 0.0, 'AMGN.US': 34.84668, 'BDX.US': 0.0, 'BMY.US': -1.0, 'HUM.US': 28.85124, 'JNJ.US': 0.0, 'LLY.US': -1.0, 'MDT.US': 3.358582, 'MRK.US': 0.0, 'PFE.US': 0.0, 'SYK.US': -1.0, 'TMO.US': -1.0, 'UNH.US': -1.0}
reward MA {'ABT.US': -1.0, 'AMGN.US': -44.997883, 'BDX.US': 0.0, 'BMY.US': -1.0, 'HUM.US': 98.261314, 'JNJ.US': 0.0, 'LLY.US': 0.0, 'MDT.US': 27.629484, 'MRK.US': -1.0, 'PFE.US': -1.0, 'SYK.US': 0.0, 'TMO.US': 0.0, 'UNH.US': 36.317}
reward MA {'ABT.US': 0.0, 'AMGN.US': -82.33981, 'BDX.US': 94.84883, 'BMY.US': 86.29734, 'HUM.US': 109.26935, 'JNJ.US': 22.216982, 'LLY.US': -1.0, 'MDT.US': 132.59062, 'MRK.US': 6.4284773, 'PFE.US': 0.0, 'SYK.US': -19.081001, 'TMO.US': -1.0, 'UNH.US': 98.82188}
reward MA {'ABT.US': -498.09515, 'AMGN.US': 86.69296, 'BDX.US': -92.58357, 'BMY.US': -384.18314, 'HUM.US': -624.4524, 'JNJ.US': -32.2925, 'LLY.US': -50.751682, 'MDT.US': -58.086876, 'MRK.US': -22.47821, 'PFE.US': 0.0, 'SYK.US': -8.643456, 'TMO.US': -1.0, 'UNH.US': -11.

KeyboardInterrupt: 

In [ ]:
if os.name == 'nt':
    path_to_save = "C:\\GitHub\\ray\\workertune"
else:
    path_to_save = "/Volumes/SSD980/ray/workertune1"


env = WorkerStandAlone()
def env_creator(env_config):
    return WorkerStandAlone()  

register_env("worker", env_creator)

asha_scheduler = ASHAScheduler(
    time_attr='training_iteration',
    max_t=5000,
    grace_period=200,
    reduction_factor=3,
    brackets=1,
)

param_space = {
    "env": "worker",
    "rollout_fragment_length": "auto",
    "framework": "tf2",
    "lr": tune.uniform(1e-5,1e-4),
    "gamma": tune.uniform(0.95, 0.9999),
    "lambda": tune.uniform(0.9,1.0),
    "entropy_coeff": tune.uniform(0.01,0.1),
    "vf_loss_coeff": tune.uniform(0.1,0.3),
    "num_workers": 2, 
    "log_level": "ERROR",
    "output": "logdir",
    "monitor": False,
}

analysis = tune.run(
    "A2C", 
    metric="episode_reward_mean", 
    num_samples=10,
    resume=False,
    mode="max",
    config=param_space, 
    local_dir=path_to_save,
    search_alg=None,
    scheduler=asha_scheduler,
    progress_reporter=CLIReporter(max_progress_rows=10,max_report_frequency=120),
    max_concurrent_trials=2,
    #checkpoint_config not checked yet
    checkpoint_config={
        "num_to_keep": 3,
        "checkpoint_score_attribute": "episode_reward_mean",
        "checkpoint_score_order": "max",
        "checkpoint_frequency": 10
    }
    )

print(pretty_print(analysis.last_result))
print("Best hyperparameters found were: ", pretty_print(analysis.best_config))

In [ ]:
env = WorkerStandAlone()
def env_creator(env_config):
    return WorkerStandAlone()  

register_env("worker", env_creator)



analysis = tune.run(
        "A2C",
        metric="episode_reward_mean",
        mode="max",
        config={
            "env": "worker",
            "env_config": {"initial_capital": 1e6},
            "lr": tune.uniform(1e-5, 1e-4),
            "train_batch_size": tune.choice([10000, 20000, 40000]),
}, )

In [ ]:


n_iterations = 30000


env = WorkerStandAlone(data=data)

state = env.reset()

for _ in range(n_iterations):
    action = env.action_space.sample()  # Generate random actions for the manager
    worker_dones = {...}  # You need to provide the values for worker_dones
    worker_truncateds = {...}  # You need to provide the values for worker_truncateds

    obs, reward, done, truncated, info = env.step(action)

    if done:
        print("Episode finished!")
        state = env.reset()
    else:
        state = obs

In [ ]:
#Mit vielen policies
import sys
# Redirect the standard error to the log file

# Create a log file
log_file = open("console_logs.txt", "w")


# Redirect the standard output to the log file
sys.stdout = log_file


env = HRL()
def env_creator(env_config):
    return HRL(env_config)  

register_env("hrl", env_creator)

def create_policy_spec(worker_id):
    # print(f"Creating policy for {worker_id} with obs space {env.observation_space[worker_id]} and action space {env.action_space[worker_id]}")
    return PolicySpec(
        observation_space=env.observation_space[worker_id],
        action_space=env.action_space[worker_id],
        config={}
    )

manager_policy_spec = PolicySpec(
    observation_space=env.observation_space['manager'],
    action_space=env.action_space['manager'],
    config={}
)

policies = {
    "manager_policy": manager_policy_spec,
}

for worker_id in env.workers:
    policies[worker_id] = create_policy_spec(worker_id)

def policy_mapping_fn(agent_id, episode=None, worker=None, **kwargs):
    if agent_id == 'manager':
        # print(f"!!!!!! policy mapping manager: {agent_id}")
        return "manager_policy"
    elif agent_id in env.workers:
        return agent_id
    else:
        print("defaul policy triggered")
        return "default_policy"

param_space = {
     "env": "hrl",
    "multiagent": {
            "policies": policies,
            "policy_mapping_fn": policy_mapping_fn,
        },
        "rollout_fragment_length": "auto",
        "framework": "tf2",
        "lr": tune.uniform(1e-5,1e-4),
        "gamma": tune.uniform(0.95, 0.9999),
        "lambda": tune.uniform(0.9,1.0),
        "entropy_coeff": tune.uniform(0.01,0.1),
        "vf_loss_coeff": tune.uniform(0.1,0.3),
        "num_workers": 3, 
        #Change for Debugging
        "log_level": "DEBUG",
        "output": "logdir",
        "monitor": True,
}

analysis = tune.run(
    "A2C",
    metric="episode_reward_mean", 
    num_samples=5,
    mode="max",
    config=param_space, 
    local_dir="/Volumes/SSD980/ray/results/tunerun2",
    search_alg=None,
    scheduler=None,
    progress_reporter=CLIReporter(max_progress_rows=5,max_report_frequency=120),
    max_concurrent_trials=2,
    #checkpoint_config not checked yet
    checkpoint_config={
        "num_to_keep": 1,
        "checkpoint_score_attribute": "episode_reward_mean",
        "checkpoint_score_order": "max",
        "checkpoint_frequency": 10
    }
)

In [ ]:
env = HRL()

def env_creator(env_config):
    return HRL(env_config)  

register_env("hrl", env_creator)

def policy_mapping_fn(agent_id, episode=None, worker=None, **kwargs):
    if agent_id in env.workers:
        return "worker_policy"
    else:
        return "manager_policy"
    
first_worker_tic = next(iter(env.workers))
worker_policy_spec = PolicySpec(
    observation_space=env.observation_space[first_worker_tic],
    action_space=env.action_space[first_worker_tic],
    config={}
)
 
manager_policy_spec = PolicySpec(
    observation_space=env.observation_space['manager'],
    action_space=env.action_space['manager'],
    config={}
)

policies = {
    "worker_policy": worker_policy_spec,
    "manager_policy": manager_policy_spec,
}
 
initial_params = [{
    "lr": 0.001,
    "gamma": 0.92,
    "lambda": 0.95,
    "entropy_coeff": 1e-3,
    "vf_loss_coeff": 0.5,
    "model": {
        "fcnet_hiddens": 64,
        "fcnet_activation":"relu",
    },
}]

search_space = {
    "lr": tune.loguniform(1e-4, 1e-1),
    "gamma": tune.uniform(0.9, 0.99),
    "lambda": tune.uniform(0.9, 1.0),
    "entropy_coeff": tune.loguniform(1e-4, 1e-1),
    "vf_loss_coeff": tune.uniform(0.1, 1.0),
    # "model": {
    #     "fcnet_hiddens": tune.grid_search([[64, 64], [128, 128], [256, 256]]),
    #     "fcnet_activation": tune.choice(["relu", "tanh"]),
    # },
}


algo = HyperOptSearch(space=search_space,metric="episode_reward_mean", mode="max",)


param_space = {
     "env": "hrl",
        "multiagent": {
            "policies": policies,
            "policy_mapping_fn": policy_mapping_fn,
        },

        "num_workers": 1,  
        "num_cpus_per_trial": 1,
         "lr": tune.loguniform(1e-4, 1e-1),
    "gamma": tune.uniform(0.9, 0.99),
    "lambda": tune.uniform(0.9, 1.0),
    "entropy_coeff": tune.loguniform(1e-4, 1e-1),
    "vf_loss_coeff": tune.uniform(0.1, 1.0),
}



param_space = {
     "env": "hrl",
        "multiagent": {
            "policies": policies,
            "policy_mapping_fn": policy_mapping_fn,
        },
          "rollout_fragment_length": "auto",
          "lr": tune.uniform(1e-5,1e-4),
          "lambda": tune.uniform(0.95, 1.0),
          "vf_loss_coeff": tune.uniform(0.1, 1.0),
          "entropy_coeff": tune.uniform(1e-4, 1e-1),
        "num_workers": 1,  
        "num_envs_per_worker": 1
}

analysis = tune.run("A2C", 
                    metric="episode_reward_mean", 
                    mode="max",
                    config=param_space,
                    num_samples=10,
                    stop={"training_iteration": 100},
                    progress_reporter=CLIReporter(max_progress_rows=10,max_report_frequency=600),
                    # local_dir="/Users/floriankockler/rayresults/overnight1",
                    storage_path="/Users/floriankockler/Documents/GitHub.nosync/raystorage",
                    checkpoint_config=CheckpointConfig(
                        num_to_keep=2,
                        checkpoint_score_attribute="episode_reward_mean", 
                        checkpoint_score_order="max"
                        )
                    )


best = analysis.best_trial
print(pretty_print(best.last_result))

In [ ]:

env = HRL()

n_iterations = 1

state = env.reset()

for _ in range(n_iterations):

    action = env.action_space.sample()

    obs, reward, done, truncated, info= env.step(action)

    # print(f"Action: {action}, Reward: {reward}, Portfolio Value: {obs[0] + obs[1] * obs[2]}")
    
    if done["__all__"]:
        print("Episode finished!")
        state = env.reset()
    else:
        state = obs

In [ ]:
def policy_mapping_fn(agent_id, episode=None, worker=None, **kwargs):
    if agent_id in env.workers:
        return "worker_policy"
    else:
        return "manager_policy"
    
first_worker_tic = next(iter(env.workers))
worker_policy_spec = PolicySpec(
    observation_space=env.observation_space[first_worker_tic],
    action_space=env.action_space[first_worker_tic],
    config={}
)
 
manager_policy_spec = PolicySpec(
    observation_space=env.observation_space['manager'],
    action_space=env.action_space['manager'],
    config={}
)
 



policies = {
    "worker_policy": worker_policy_spec,
    "manager_policy": manager_policy_spec,
}

    
algo = PPOConfig().environment(env=HRL).multi_agent(
        policies=policies,
        policy_mapping_fn = policy_mapping_fn,  
    ).training(train_batch_size=4000).build()

print(algo.train())

In [ ]:
env = HRL()

def env_creator(env_config):
    return HRL(env_config)  

register_env("hrl", env_creator)

def policy_mapping_fn(agent_id, episode=None, worker=None, **kwargs):
    if agent_id in env.workers:
        return "worker_policy"
    else:
        return "manager_policy"
    
first_worker_tic = next(iter(env.workers))
worker_policy_spec = PolicySpec(
    observation_space=env.observation_space[first_worker_tic],
    action_space=env.action_space[first_worker_tic],
    config={}
)
 
manager_policy_spec = PolicySpec(
    observation_space=env.observation_space['manager'],
    action_space=env.action_space['manager'],
    config={}
)

policies = {
    "worker_policy": worker_policy_spec,
    "manager_policy": manager_policy_spec,
}
 
search_space = {
    "lr": tune.loguniform(1e-4, 1e-1),  # Learning rate
    "gamma": tune.uniform(0.9, 0.99),  # Discount factor
}

tune_config = TuneConfig(
    metric="episode_reward_mean",
    mode="max",
    max_concurrent_trials=1,
    num_samples=100,
    search_alg=BayesOptSearch(
          search_space,
        metric="episode_reward_mean",
        mode="max",
        random_search_steps=4
    ),
    scheduler=HyperBandScheduler()
)

run_config = RunConfig(
    name="MyExperiment",
    storage_path="/Users/floriankockler/rayresults/tuner_trial",
    verbose=2,
    # checkpoint_config=air.CheckpointConfig(checkpoint_frequency=2),
    callbacks=None,
    stop=None,
    failure_config=None,
    sync_config=None,
    checkpoint_config=None,
    progress_reporter=CLIReporter(max_progress_rows=10),# Define the policy mapping function

)

param_space = {
     "env": "hrl",
        "multiagent": {
            "policies": policies,
            "policy_mapping_fn": policy_mapping_fn,
        },
          "rollout_fragment_length": "auto",
          "lr": tune.uniform(1e-5,1e-4),
        "num_workers": 1,  
        "num_cpus_per_trial": 1,
}

analysis = tune.run("A2C", metric="episode_reward_mean", mode="max",config=param_space)

In [ ]:
from ray.rllib.algorithms.ppo import PPOConfig
algo = PPOConfig().environment(env=HRL).multi_agent(
    policies={
        "policy_1": ()
    }
)

In [ ]:
from ray.tune import CLIReporter
def env_creator(env_config):
    return HRL(env_config)  # Assuming this is your environment
 
env = HRL()

register_env("hrl", env_creator)

def policy_mapping_fn(agent_id, episode=None, worker=None, **kwargs):
    if agent_id in env.workers:
        return "worker_policy"
    else:
        return "manager_policy"
    
first_worker_tic = next(iter(env.workers))
worker_policy_spec = PolicySpec(
    observation_space=env.observation_space[first_worker_tic],
    action_space=env.action_space[first_worker_tic],
    config={}
)
 
manager_policy_spec = PolicySpec(
    observation_space=env.observation_space['manager'],
    action_space=env.action_space['manager'],
    config={}
)

policies = {
    "worker_policy": worker_policy_spec,
    "manager_policy": manager_policy_spec,
}
 
search_space = {
    "lr": tune.loguniform(1e-4, 1e-1),  # Learning rate
    "gamma": tune.uniform(0.9, 0.99),  # Discount factor
}

tune_config = TuneConfig(
    metric="episode_reward_mean",
    mode="max",
    max_concurrent_trials=1,
    num_samples=100,
    search_alg=BayesOptSearch(
          search_space,
        metric="episode_reward_mean",
        mode="max",
        random_search_steps=4
    ),
    scheduler=HyperBandScheduler()
)

run_config = RunConfig(
    name="MyExperiment",
    storage_path="/Users/floriankockler/rayresults/tuner_trial",
    verbose=2,
    # checkpoint_config=air.CheckpointConfig(checkpoint_frequency=2),
    callbacks=None,
    stop=None,
    failure_config=None,
    sync_config=None,
    checkpoint_config=None,
    progress_reporter=CLIReporter(max_progress_rows=10),# Define the policy mapping function

)

param_space = {
     "env": "hrl",
        "multiagent": {
            "policies": policies,
            "policy_mapping_fn": policy_mapping_fn,
        },
          "rollout_fragment_length": "auto",
        "num_workers": 1,  
        "num_cpus_per_trial": 1,
}

tuner = tune.Tuner(
    "A2C",
    tune_config=tune_config,
    param_space=param_space,

    run_config=run_config,
)
results = tuner.fit()


In [ ]:
import random
import os
from ray import air, tune
from ray.tune.schedulers import PopulationBasedTraining
import argparse
from ray.tune import CLIReporter
from env.multi_agent.hrl import HRL

reporter = CLIReporter(max_progress_rows=10)

def env_creator(env_config):
    return HRL(env_config)  # Assuming this is your environment
 
register_env("hrl", env_creator)

manager_config = {
    "df": train_df,

}
hrl_config={
        "manager_config": manager_config
        }
env = HRL(hrl_config)
 
def policy_mapping_fn(agent_id, episode=None, worker=None, **kwargs):
    if agent_id in env.workers:
        return "worker_policy"
    else:
        return "manager_policy"
 


first_worker_tic = next(iter(env.workers))
worker_policy_spec = PolicySpec(
    observation_space=env.observation_space[first_worker_tic],
    action_space=env.action_space[first_worker_tic],
    config={}
)
 
manager_policy_spec = PolicySpec(
    observation_space=env.observation_space['manager'],
    action_space=env.action_space['manager'],
    config={}
)
 

parser = argparse.ArgumentParser()
parser.add_argument(
    "--smoke-test", action="store_true", help="Finish quickly for testing"
)
args, _ = parser.parse_known_args()

policies = {
    "worker_policy": worker_policy_spec,
    "manager_policy": manager_policy_spec,
}


def explore(config):
    # Ensure we collect enough timesteps to do sgd
    if config["train_batch_size"] < config["rollout_fragment_length"] * 2:
        config["train_batch_size"] = config["rollout_fragment_length"] * 2
    return config

hyperparam_mutations = {
    "lr": [1e-3, 5e-4, 1e-4, 5e-5, 1e-5],
    "gamma": lambda: random.uniform(0.9, 1.0),
    "entropy_coeff": [0.01, 0.1, 1.0],
    "num_envs_per_worker": [1, 2, 4, 8],
    #"rollout_fragment_length": [50, 100, 200, 400],
    "train_batch_size": lambda: random.randint(200, 1500),
    "sgd_minibatch_size": tune.choice([50, 100, 200]),

}

pbt = PopulationBasedTraining(
        time_attr="time_total_s",
        perturbation_interval=120,
        resample_probability=0.25,
        # Specifies the mutations of these hyperparams
        hyperparam_mutations=hyperparam_mutations,
        custom_explore_fn=explore,
    )

# Stop when we've reached 100 training iterations or reward=300
stopping_criteria = {"training_iteration": 100}

tuner = tune.Tuner(
    "A2C",
    tune_config=tune.TuneConfig(
        metric="episode_reward_mean",
        mode="max",
        scheduler=pbt,
        num_samples=1 if args.smoke_test else 10,
    ),
    param_space={
        "env": "hrl",
        "multiagent": {
            "policies": policies,
            "policy_mapping_fn": policy_mapping_fn,
        },
        "env_config": hrl_config,
        "rollout_fragment_length": "auto",
        "framework": "tf2",
        "num_workers": 1,  # 1 for training + 4 for sampling
        "num_cpus_per_trial": 3,
        # "num_cpus": 1,  # number of CPUs to use per trial --> 6 in total = max available
        # "num_gpus": 0,  # number of GPUs to use per trial
        # These params are tuned from a fixed starting value.
        "lr": 1e-4,
        # These params start off randomly drawn from a set.
        "sgd_minibatch_size": tune.choice([50, 100, 200]),
        "train_batch_size": tune.choice([200, 400, 600]),
    },

    run_config=air.RunConfig(stop=stopping_criteria, local_dir="/Users/floriankockler/rayresults/autobatch", progress_reporter=reporter),
)
results = tuner.fit()